# Synthetic Aperature Radar (SAR) Satellite Location Demo

## Summary

This notebook visualizes select SAR satellite orbit paths over Myanmar and Thailand from when the code is run until 48h later, to aid in humanitarian response to the region.

Please click on `Run` in the menu bar, followed by `Run All Cells`. You'll see an `*` next to the cells that are currently or waiting to be run, and a number once the cell has run. If you see a blank bracket `[ ]`, please re-run all cells again. 

If the last cell doesn't show a map, please re-run that cell by selecting it and then pressing `shift + enter`

Cells have been hidden to focus on the output. Please click on any section or cell to expand it.

## Background

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians and emergency response organizations, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians emergency response organizations better understand where aid is most needed.

This notebook demonstrates how the location of synthetic aperature radar (SAR) EO satellites can be computed in an area of interest.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics. Shapely will help us with some of the geometry creation and calculations, and Lonboard will be used to visualize the results.

In [ ]:
import micropip
await micropip.install("https://ds-wheels.s3.amazonaws.com/sgp4-2.23-cp312-cp312-pyodide_2024_0_wasm32.whl")

%pip install pyodide-unix-timezones

In [ ]:
deps = [
    "https://ds-wheels.s3.amazonaws.com/pyarrow-17.0.0-cp312-cp312-pyodide_2024_0_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_core-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_compute-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_io-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/geoarrow_rust_core-0.3.0b1-cp38-abi3-emscripten_3_1_58_wasm32.whl",
    "palettable",
    "matplotlib",
    "lonboard==0.10.0b2"
]
await micropip.install(deps)

In [ ]:
%pip install skyfield

In [ ]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, Polygon

import requests

## Set up list of satellites, area of interest (AOI) and timeframe

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

This demo fetches the orbital predictions for some (but not all) SAR satellites.

In [ ]:
urls = [
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2014-016", # SENTINEL-1A
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-235", # SENTINEL-2C
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2007-061", # RADARSAT-2
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2012-017" # RISAT-1
]

# Output file to store all TLE data
with open("combined_tle.txt", "w") as outfile:
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            outfile.write(response.text.strip() + "\n")  # Add double newline between TLE blocks
        else:
            print(f"Failed to fetch {url} (status: {response.status_code})")

In [ ]:
satellites = load.tle_file("combined_tle.txt", reload=True, filename="satellites")
satellites # this shows the full list of satellites whose position is being calculated.

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +2 days (48h) from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run). These techniques can also be applied in the past, if a before/after analysis is needed.

In [ ]:
# Calculate the current UTC time (without microseconds), then creating a time range + 48h

ts = load.timescale()

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2) # change this if you'd like to have a different timeframe

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [ ]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords

Let's refine our area of interest to see when/where satellites pass overhead. This demo focuses on the area surrounding the earthquake in Myanmar in March 2024

In [ ]:
min_lon = x_min = 88
max_lon = x_max = 108
min_lat = y_min = 7
max_lat = y_max = 28

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

# Defining our AOI as a polygon shape
area = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max), (x_min,y_min)])

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [ ]:
timer = t1

rows = []

while timer <= t2:
    for sat in satellites:
        timer, longitude, latitude, coords = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=1) # ~ 10s

satellites_df = pd.concat(rows, ignore_index=True)
satellites_df["time_string"] = satellites_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [ ]:
satellites_gdf = gpd.GeoDataFrame(satellites_df, geometry="coordinates")

We only want to look at the satellite pass overs within our AOI.

In [ ]:
mask_lon = (satellites_gdf.lng >= min_lon) & (satellites_gdf.lng <= max_lon)
mask_lat = (satellites_gdf.lat >= min_lat) & (satellites_gdf.lat <= max_lat)

aoi = satellites_gdf.where(mask_lon & mask_lat).dropna()

## Visualize path of satellites

We can "connect the dots" calculated in the previous step to show the path the satellite takes

In [ ]:
def create_individual_linestrings(coords, timestamps, time_strings):
    coords_list = coords.tolist()
    timestamps_list = timestamps.tolist()
    time_strings_list = time_strings.tolist()
    line_segments = [
        {'linestring': LineString([coords_list[i], coords_list[i + 1]]), 'timestamp': timestamps_list[i], 'time_string': time_strings_list[i]} 
        for i in range(len(coords_list) - 1)
    ]
    return line_segments

In [ ]:
new_rows = []

for satellite, group in aoi.groupby('satellite'):
    line_segments = create_individual_linestrings(group['coordinates'], group['timestamp'], group['time_string'])
    for segment in line_segments:
        new_rows.append({'satellite': satellite, 'linestring': segment['linestring'], 'timestamp': segment['timestamp'], 'time_string': segment['time_string']})

path_segments = gpd.GeoDataFrame(new_rows, geometry='linestring')

If these line segments are not within the AOI, we can drop them from the dataframe.

In [ ]:
def is_within_aoi(linestring, aoi):
    return linestring.intersects(aoi)

In [ ]:
path_segments['within_aoi'] = path_segments['linestring'].apply(lambda x: is_within_aoi(x, area))
path_segments_clipped = path_segments[path_segments['within_aoi']].drop(columns='within_aoi')

path_segments_clipped['length'] = path_segments_clipped['linestring'].apply(lambda x: x.length)

# This is a way to drop segments that are not proper representations of paths, such as errors caused by traversing the International Date Line
path_segments_clipped = path_segments_clipped[path_segments_clipped["length"] <= 5].drop(columns='length')

As with the single satellite, we can create a color scale that shows the time of the satellite's pass for each line segment. In this example, white is closer to `now` and orange is closer to +48h.

In [ ]:
# This creates a range from 0-1 to define our colormap.
time_norm_satellites = (path_segments_clipped.timestamp - t1) / (t2 - t1)

colors_satellites = apply_continuous_cmap(time_norm_satellites, Oranges_9, alpha=.5)

We can roughly estimate the width of the ground coverage of a satellite to be 10km (i.e., the width of an image is 6km). To visualize this, we can set `get_width` to be 10km.

In [ ]:
layer2 = PathLayer.from_geopandas(
    path_segments_clipped,
    get_color = colors_satellites,
    get_width=6000,
    opacity=1,
    auto_highlight=True
)

# Map

In [ ]:
m2 = Map(
    [layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2